In [1]:
from estnltk import Text
import csv
from datetime import datetime
from tqdm import tqdm
import psycopg2
import re

In [ ]:
from cda_data_cleaning.common.db_operations import create_connection
from cda_data_cleaning.common import read_config

In [2]:
config_file = 'egcut_epi.ini'
config = read_config(config_file)

conn = create_connection(config)
cur = conn.cursor()

In [3]:
def clean_date(date_str):
    '''
    Turn datestring into datetime object
    '''
    if date_str:
        stripped_date = date_str.strip('\t\n,a ')
        stripped_date = stripped_date.replace(' ', '.')
        stripped_date = stripped_date.replace('..', '.')
    else:
        return
    if len(stripped_date) == 8:
        date = datetime.strptime(stripped_date, '%Y%m%d')
        return date
    elif len(stripped_date) == 9 and re.match('[0-9]\.', stripped_date):
        stripped_date_fixed = '0' + stripped_date
        date = datetime.strptime(stripped_date_fixed, '%d.%m.%Y')
        return date
    elif len(stripped_date) == 10:
        date = datetime.strptime(stripped_date, '%d.%m.%Y')
        return date
    else:
        return 

In [4]:
cur.execute("""set role hwisc_epi_work_create;""")

In [5]:
#cur.execute("""drop table work.drug_parsed_v0;""")

ProgrammingError: table "drug_parsed_v0" does not exist


In [5]:
cur.execute("""create table work.drug_parsed_v0 (id bigint, epi_id text, 
epi_type text, date_raw text, date date, ATC text, drug_name text, active_ingredient text);""")

In [6]:
with open("parsed_drugs.csv", "r") as fin:
    reader = csv.reader(fin)
    for idx, row in enumerate(reader):
        if idx%100000 == 0:
            print(idx)
        new_date = clean_date(row[3])
        cur.execute("insert into work.drug_parsed_v0 VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",  (row[0], row[1], row[2], row[3], new_date, row[4], row[5], row[6]))

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000


In [7]:
conn.commit()
cur.close()
conn.close()